### General information

This kernel is intented to help mlcourse.ai participants with doing EDA, Feature Engineering and building models.

* At first I'll do basic EDA of the data;
* After this I'll build a baseline model to see how good model can be on the basic data;
* Then I'll create new features based on the main features and train a model again to see whether there is an improvement;
* After this I'll try to extract new features from json files and see whether it helps;

In [10]:
# Libraries
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import collections
import datetime
import json
import ast
import time
from scipy import stats
import os
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import metrics
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import GenericUnivariateSelect, SelectPercentile, SelectKBest, f_classif, mutual_info_classif, RFE
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

import xgboost as xgb
import lightgbm as lgb
import eli5
from eli5.sklearn import PermutationImportance
import shap
from tqdm import tqdm_notebook
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')
from catboost import CatBoostClassifier
from itertools import combinations


# import json
import altair as alt
from  altair.vega import v3
from IPython.display import HTML
from IPython.display import display_html
!pip install ujson
import ujson as json

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import warnings
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

SEED = 42

In [2]:

class ColumnDataProcessor:
    to_scale = True

    def replaceNaNValues(self, A):
        A[np.isnan(A)] = 0
        A[np.isinf(A)] = 0
        return A

    def add_team_features(self, df, c, r_columns, d_columns):
        drop_features = []
        # df['r_total_' + c] = df[r_columns].sum(1)
        # df['d_total_' + c] = df[d_columns].sum(1)
        # df['total_' + c + '_ratio'] = df['r_total_' + c] / df['d_total_' + c]
        # df['total_' + c + '_ratio'] = self.replaceNaNValues(df['total_' + c + '_ratio'])
        # drop_features = drop_features + ['r_total_' + c, 'd_total_' + c]

        df['r_std_' + c] = df[r_columns].std(1)
        df['d_std_' + c] = df[d_columns].std(1)
        df['std_' + c + '_ratio'] = df['r_std_' + c] / df['d_std_' + c]
        df['std_' + c + '_ratio'] = self.replaceNaNValues(df['std_' + c + '_ratio'])
        drop_features = drop_features + ['r_std_' + c, 'd_std_' + c]

        df['r_mean_' + c] = df[r_columns].mean(1)
        df['d_mean_' + c] = df[d_columns].mean(1)
        df['mean_' + c + '_ratio'] = df['r_mean_' + c] / df['d_mean_' + c]
        df['mean_' + c + '_ratio'] = self.replaceNaNValues(df['mean_' + c + '_ratio'])
        # drop_features = drop_features + ['r_mean_' + c, 'd_mean_' + c]

        df = df.drop(r_columns, axis=1)
        df = df.drop(d_columns, axis=1)
        df = df.drop(drop_features, axis=1)
        return df

    # As we see coordinate features (x and y) are quite important. However, I think we need to combine them into one
    # feature.Simplest idea is the distance from the left bottom corner. So, short distances mean near own base,
    # long distances - near the enemy base
    def make_coordinate_features(self, df):
        for team in 'r', 'd':
            players = [f'{team}{i}' for i in range(1, 6)]  # r1, r2...
            for player in players:
                df[f'{player}_distance'] = np.sqrt(df[f'{player}_x'] ** 2 + df[f'{player}_y'] ** 2)
                df.drop(columns=[f'{player}_x', f'{player}_y'], inplace=True)
        return df

    # def hot_feat_hero_id(self, df):
    #     for team in ['r', 'd']:
    #         for i in range(1, 6):
    #             df = pd.concat([df, pd.get_dummies(df[f'{team}{i}_hero_id'], prefix=f'{team}{i}_hero_id')], axis=1)
    #
    #     return df

    def hero_id_subset_analyzer(self, text):
        # it takes a string of hero ids (like '1 2 5 4 3') as input
        ids = set()
        for i in range(1, 4):  # we need all subset of lenght 1-3. I think longer combinations are not relevant
            hero_ids = text.split(' ')  # '1 2 5 4 3'-> ['1', '2', '5', '4', '3']
            hero_ids.sort()  # sort them as '1 2 5 4 3' and '3 1 4 5 3' should produce the same set of tokens
            combs = set(
                combinations(hero_ids, i))  # all combinations of length i e.g for 2 are: (1,2), (1,3)... (2,5)... etc
            ids = ids.union(combs)
        ids = {"_".join(item) for item in ids}  # convert from lists to string e.g. (1,2) -> '1_2'
        return ids

    def replace_hero_ids(self, train, test):
        vectorizer = TfidfVectorizer(self.hero_id_subset_analyzer, ngram_range=(1, 1), max_features=1000,
                                     tokenizer=lambda s: s.split())
        train = self.replace_hero_ids_df(train, vectorizer)
        test = self.replace_hero_ids_df(test, vectorizer, train=False)
        return train, test

    def replace_hero_ids_df(self, df, vectorizer, train=True):

        # ngram range is (1,1) as all combinations are created by analyser
        # 1000 features - I think it's enough to cover all heroes + popular combos

        for team in 'r', 'd':
            players = [f'{team}{i}' for i in range(1, 6)]  # r1, r2,...
            hero_columns = [f'{player}_hero_id' for player in players]  # r1_hero_id,....

            # combine all hero id columns into one
            df_hero_id_as_text = df[hero_columns].apply(lambda row: ' '.join([str(i) for i in row]), axis=1).tolist()

            if train:
                new_cols = pd.DataFrame(vectorizer.fit_transform(df_hero_id_as_text).todense(),
                                        columns=vectorizer.get_feature_names())
            else:
                new_cols = pd.DataFrame(vectorizer.transform(df_hero_id_as_text).todense(),
                                        columns=vectorizer.get_feature_names())

            # add index to vectorized dataset - needed for merge?
            new_cols['match_id_hash'] = df.index.values
            new_cols = new_cols.set_index('match_id_hash').add_prefix(f'{team}_hero_')  # e.g.r_hero_10_21

            # df = pd.merge(df, new_cols)
            df = pd.merge(df, new_cols, on='match_id_hash')
            df.drop(columns=hero_columns, inplace=True)

        return df

    def prepare_data(self, train, target, test, features_list):
        print('prepare_data.. Start')

        r_heroes = [f'r{i}_hero_id' for i in range(1, 6)]
        d_heroes = [f'd{i}_hero_id' for i in range(1, 6)]

        train = self.make_coordinate_features(train)
        test = self.make_coordinate_features(test)
        # As the distance is also a numeric feature convert it into the team features
        features_list = features_list + ['distance']

        print('prepare_data.. Adding team features')
        for c in features_list:
            r_columns = [f'r{i}_{c}' for i in range(1, 6)]
            d_columns = [f'd{i}_{c}' for i in range(1, 6)]

            train = self.add_team_features(train, c, r_columns, d_columns)
            test = self.add_team_features(test, c, r_columns, d_columns)

            if self.to_scale:
                features_to_scale = \
                    ['std_' + c + '_ratio', 'mean_' + c + '_ratio', 'r_mean_' + c, 'd_mean_' + c]
                #    ['total_' + c + '_ratio', 'std_' + c + '_ratio', 'mean_' + c + '_ratio']  # + r_heroes + d_heroes
                scaler = MinMaxScaler()
                train[features_to_scale] = scaler.fit_transform(train[features_to_scale])
                test[features_to_scale] = scaler.transform(test[features_to_scale])

        print('prepare_data.. Replace heroes id')

        train, test = self.replace_hero_ids(train, test)
        # train = self.hot_feat_hero_id(train)
        # test = self.hot_feat_hero_id(test)

        feat_to_drop = ['game_time', 'game_mode', 'lobby_type', 'objectives_len', 'chat_len']  # + r_heroes + d_heroes
        print('prepare_data.. Drop extra columns: {}'.format(feat_to_drop))
        train = train.drop(feat_to_drop, axis=1)
        test = test.drop(feat_to_drop, axis=1)

        return self.prepare_data_simple(train, target, test)

    def prepare_data_simple(self, train, targets, test):
        X = train.reset_index(drop=True)
        y = targets['radiant_win']
        X_test = test.reset_index(drop=True)

        # for col in train.columns:
        #     if train[col].isnull().any():
        #         print(col, train[col].isnull().sum())
        #
        # for col in test.columns:
        #     if test[col].isnull().any():
        #         print(col, test[col].isnull().sum())

        print("\n\nPrepared data frame: ")
        print(X.describe())
        print('Dimensions: train {}, test {}'.format(X.shape, X_test.shape))

        return X, y, X_test


class CSVDataPrepare:

    def read_data_frame(self):
        PATH_TO_DATA = '../input/'

        # Train dataset
        df_train_features = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_features.csv'), index_col='match_id_hash')
        df_train_targets = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_targets.csv'), index_col='match_id_hash')
        # Test dataset
        df_test_features = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_features.csv'), index_col='match_id_hash')
        # Check if there is missing data
        print("Original data frame (CSV): ")
        # print('df_train_features.isnull() {}'.format(df_train_features.isnull().values.any()))
        # print('df_test_features.isnull() {}'.format(df_test_features.isnull().values.any()))
        print(df_train_features.shape)
        # print(df_train_features.index.values)
        return df_train_features, df_train_targets, df_test_features

    def prepareDataOld(self, train, target, test):
        # Let's combine train and test datasets in one dataset.
        # This allows for addding new features for both datasets at the same time.
        df_full_features = pd.concat([train, test])

        # Index to split the training and test data sets
        idx_split = train.shape[0]

        # That is,
        # df_train_features == df_full_features[:idx_split]
        # df_test_features == df_full_features[idx_split:]

        df_full_features.drop(['game_time', 'game_mode', 'lobby_type', 'objectives_len', 'chat_len'],
                              inplace=True, axis=1)

        # Clearly the hero_id is a categorical feature, so let's one-hot encode it. Note that according to wiki there are
        # 117 heroes, however in our dataset there are 116 heroes with ids 1, 2, ..., 114, 119, 120.
        # You will get the same result for all teams and players, here I use r1.
        np.sort(np.unique(df_full_features['r1_hero_id'].values.flatten()))

        for t in ['r', 'd']:
            for i in range(1, 6):
                df_full_features = pd.get_dummies(df_full_features, columns=[f'{t}{i}_hero_id'])
        #         df_full_features = pd.concat([df_full_features,
        #           pd.get_dummies(df_full_features[f'{t}{i}_hero_id'], prefix=f'{t}{i}_hero_id')], axis=1)

        # Finally let's scale the player-features that have relatively large values, such as gold, lh, xp etc.
        player_features = set(f[3:] for f in train.columns[5:])
        features_to_scale = []
        for t in ['r', 'd']:
            for i in range(1, 6):
                for f in player_features - {'hero_id', 'firstblood_claimed', 'teamfight_participation'}:
                    features_to_scale.append(f'{t}{i}_{f}')
        df_full_features_scaled = df_full_features.copy()
        df_full_features_scaled[features_to_scale] = MinMaxScaler().fit_transform(
            df_full_features_scaled[features_to_scale])

        df_full_features_scaled.head()
        df_full_features_scaled.max().sort_values(ascending=False).head(12)

        # Let's construct X and y arrays.
        X_train = df_full_features_scaled[:idx_split]
        X_test = df_full_features_scaled[idx_split:]
        y_train = target['radiant_win'].map({True: 1, False: 0})

        print(X_train.head())
        print(X_train.describe())

        # splitting whole dataset on train and test
        # X_train = data.loc[:test_index].drop(["y"], axis=1)
        # y_train = data.loc[:test_index]["y"]
        # X_test = data.loc[test_index:].drop(["y"], axis=1)
        # y_test = data.loc[test_index:]["y"]

        return X_train, X_test, y_train

    def prepareValidationTensors(self, X_train, X_test, y_train, test_size=0.2):
        # Perform a train/validation split
        X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train,
                                                                        test_size=test_size,
                                                                        random_state=SEED)

        # Convert to pytorch tensors
        X_train_tensor = torch.from_numpy(X_train_part.values).float()
        X_valid_tensor = torch.from_numpy(X_valid.values).float()
        y_train_tensor = torch.from_numpy(y_train_part.values).float()
        y_valid_tensor = torch.from_numpy(y_valid.values).float()
        X_test_tensor = torch.from_numpy(X_test.values).float()

        # Create the train and validation dataloaders
        train_dataset = data.TensorDataset(X_train_tensor, y_train_tensor)
        valid_dataset = data.TensorDataset(X_valid_tensor, y_valid_tensor)

        dataloaders = {'train': data.DataLoader(train_dataset, batch_size=1000, shuffle=True, num_workers=2),
                       'valid': data.DataLoader(valid_dataset, batch_size=1000, shuffle=False, num_workers=2)}
        return dataloaders, X_train_tensor, X_valid_tensor, y_train_tensor, y_valid_tensor, X_test_tensor

    # My idea behind this FE is the following: Let's take gold, for example. Gold earned by each player can't give
    # us a lot of information. But what is we take total gold by the team? Maybe teams with more gold earned usually
    # win. What if we take mean and std of players' gold in a team? Maybe teams where players tend to have similar
    # parameters are more likely to win. Let's try creating these features.
    FEATURES_LIST = ['kills', 'deaths', 'assists', 'denies', 'gold', 'lh', 'xp', 'health', 'max_health', 'max_mana',
                     'level', 'stuns', 'creeps_stacked', 'camps_stacked', 'rune_pickups',
                     'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'obs_placed',
                     'sen_placed']

    def prepare_data(self, train, target, test):
        engineering = ColumnDataProcessor()
        train, target, test = engineering.prepare_data(train, target, test, self.FEATURES_LIST)

        return train, target, test


class JsonDataPrepare:
    MATCH_FEATURES = [
        ('game_time', lambda m: m['game_time']),
        ('game_mode', lambda m: m['game_mode']),
        ('lobby_type', lambda m: m['lobby_type']),
        ('objectives_len', lambda m: len(m['objectives'])),
        ('chat_len', lambda m: len(m['chat'])),
    ]

    PLAYER_FIELDS = [
        'hero_id',

        'kills',
        'deaths',
        'assists',
        'denies',

        'gold',
        'lh',
        'xp',
        'health',
        'max_health',
        'max_mana',
        'level',

        'x',
        'y',

        'stuns',
        'creeps_stacked',
        'camps_stacked',
        'rune_pickups',
        'firstblood_claimed',
        'teamfight_participation',
        'towers_killed',
        'roshans_killed',
        'obs_placed',
        'sen_placed',
    ]

    def extract_features_csv(self, match):
        row = [
            ('match_id_hash', match['match_id_hash']),
        ]

        for field, f in self.MATCH_FEATURES:
            row.append((field, f(match)))

        for slot, player in enumerate(match['players']):
            if slot < 5:
                player_name = 'r%d' % (slot + 1)
            else:
                player_name = 'd%d' % (slot - 4)

            for field in self.PLAYER_FIELDS:
                column_name = '%s_%s' % (player_name, field)
                row.append((column_name, player[field]))
            row.append((f'{player_name}_ability_level', len(player['ability_upgrades'])))
            row.append((f'{player_name}_max_hero_hit', player['max_hero_hit']['value']))
            row.append((f'{player_name}_purchase_count', len(player['purchase_log'])))
            row.append((f'{player_name}_count_ability_use', sum(player['ability_uses'].values())))
            row.append((f'{player_name}_damage_dealt', sum(player['damage'].values())))
            row.append((f'{player_name}_damage_received', sum(player['damage_taken'].values())))

        return collections.OrderedDict(row)

    def extract_targets_csv(self, match, targets):
        return collections.OrderedDict([('match_id_hash', match['match_id_hash'])] + [
            (field, targets[field])
            for field in ['game_time', 'radiant_win', 'duration', 'time_remaining', 'next_roshan_team']
        ])

    def read_matches(self, matches_file):
        MATCHES_COUNT = {
            'test_matches.jsonl': 10000,
            'train_matches.jsonl': 39675,
        }
        _, filename = os.path.split(matches_file)
        total_matches = MATCHES_COUNT.get(filename)

        with open(matches_file) as fin:
            for line in tqdm_notebook(fin, total=total_matches):
                yield json.loads(line)

    def read_data_frame(self):
        PATH_TO_DATA = '../input/'
        df_new_features = []
        df_new_targets = []

        for match in self.read_matches(os.path.join(PATH_TO_DATA, 'train_matches.jsonl')):
            # match_id_hash = match['match_id_hash']
            features = self.extract_features_csv(match)
            targets = self.extract_targets_csv(match, match['targets'])

            df_new_features.append(features)
            df_new_targets.append(targets)

        df_new_features = pd.DataFrame.from_records(df_new_features).set_index('match_id_hash')
        df_new_targets = pd.DataFrame.from_records(df_new_targets).set_index('match_id_hash')

        test_new_features = []
        for match in self.read_matches(os.path.join(PATH_TO_DATA, 'test_matches.jsonl')):
            # match_id_hash = match['match_id_hash']
            features = self.extract_features_csv(match)

            test_new_features.append(features)

        test_new_features = pd.DataFrame.from_records(test_new_features).set_index('match_id_hash')

        print("Original data frame (JSON): ")
        print(df_new_features.shape)

        return df_new_features, df_new_targets, test_new_features

    FEATURES_LIST = ['kills', 'deaths', 'assists', 'denies', 'gold', 'lh', 'xp', 'health', 'max_health', 'max_mana',
                     'level', 'stuns', 'creeps_stacked', 'camps_stacked', 'rune_pickups',
                     'firstblood_claimed', 'teamfight_participation', 'towers_killed', 'roshans_killed', 'obs_placed',
                     'sen_placed', 'ability_level', 'max_hero_hit', 'purchase_count', 'count_ability_use',
                     'damage_dealt', 'damage_received']

    def prepare_data(self, train, target, test):
        engineering = ColumnDataProcessor()
        train, target, test = engineering.prepare_data(train, target, test, self.FEATURES_LIST)

        return train, target, test


data_loader_csv = CSVDataPrepare()
data_loader_json = JsonDataPrepare()
data_loader = data_loader_json
print('added')

added


In [3]:
df_train_features, df_train_targets, df_test_features = data_loader.read_data_frame();

Original data frame (CSV): 
(39675, 245)


In [4]:
# Data check
df_train_features.head()


,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,r1_gold,r1_lh,r1_xp,r1_health,r1_max_health,r1_max_mana,r1_level,r1_x,r1_y,r1_stuns,r1_creeps_stacked,r1_camps_stacked,r1_rune_pickups,r1_firstblood_claimed,r1_teamfight_participation,r1_towers_killed,r1_roshans_killed,r1_obs_placed,r1_sen_placed,r2_hero_id,r2_kills,r2_deaths,r2_assists,r2_denies,r2_gold,r2_lh,r2_xp,r2_health,r2_max_health,r2_max_mana,r2_level,r2_x,r2_y,r2_stuns,r2_creeps_stacked,r2_camps_stacked,r2_rune_pickups,r2_firstblood_claimed,r2_teamfight_participation,r2_towers_killed,r2_roshans_killed,r2_obs_placed,r2_sen_placed,r3_hero_id,r3_kills,r3_deaths,r3_assists,r3_denies,r3_gold,r3_lh,r3_xp,r3_health,r3_max_health,r3_max_mana,r3_level,r3_x,r3_y,r3_stuns,r3_creeps_stacked,r3_camps_stacked,r3_rune_pickups,r3_firstblood_claimed,r3_teamfight_participation,r3_towers_killed,r3_roshans_killed,r3_obs_placed,r3_sen_placed,r4_hero_id,r4_kills,r4_deaths,r4_assists,r4_denies,r4_gold,r4_lh,r4_xp,r4_health,r4_max_health,r4_max_mana,r4_level,r4_x,r4_y,r4_stuns,r4_creeps_stacked,r4_camps_stacked,r4_rune_pickups,r4_firstblood_claimed,r4_teamfight_participation,r4_towers_killed,r4_roshans_killed,r4_obs_placed,r4_sen_placed,r5_hero_id,r5_kills,r5_deaths,r5_assists,r5_denies,r5_gold,r5_lh,r5_xp,r5_health,r5_max_health,r5_max_mana,r5_level,r5_x,r5_y,r5_stuns,r5_creeps_stacked,r5_camps_stacked,r5_rune_pickups,r5_firstblood_claimed,r5_teamfight_participation,r5_towers_killed,r5_roshans_killed,r5_obs_placed,r5_sen_placed,d1_hero_id,d1_kills,d1_deaths,d1_assists,d1_denies,d1_gold,d1_lh,d1_xp,d1_health,d1_max_health,d1_max_mana,d1_level,d1_x,d1_y,d1_stuns,d1_creeps_stacked,d1_camps_stacked,d1_rune_pickups,d1_firstblood_claimed,d1_teamfight_participation,d1_towers_killed,d1_roshans_killed,d1_obs_placed,d1_sen_placed,d2_hero_id,d2_kills,d2_deaths,d2_assists,d2_denies,d2_gold,d2_lh,d2_xp,d2_health,d2_max_health,d2_max_mana,d2_level,d2_x,d2_y,d2_stuns,d2_creeps_stacked,d2_camps_stacked,d2_rune_pickups,d2_firstblood_claimed,d2_teamfight_participation,d2_towers_killed,d2_roshans_killed,d2_obs_placed,d2_sen_placed,d3_hero_id,d3_kills,d3_deaths,d3_assists,d3_denies,d3_gold,d3_lh,d3_xp,d3_health,d3_max_health,d3_max_mana,d3_level,d3_x,d3_y,d3_stuns,d3_creeps_stacked,d3_camps_stacked,d3_rune_pickups,d3_firstblood_claimed,d3_teamfight_participation,d3_towers_killed,d3_roshans_killed,d3_obs_placed,d3_sen_placed,d4_hero_id,d4_kills,d4_deaths,d4_assists,d4_denies,d4_gold,d4_lh,d4_xp,d4_health,d4_max_health,d4_max_mana,d4_level,d4_x,d4_y,d4_stuns,d4_creeps_stacked,d4_camps_stacked,d4_rune_pickups,d4_firstblood_claimed,d4_teamfight_participation,d4_towers_killed,d4_roshans_killed,d4_obs_placed,d4_sen_placed,d5_hero_id,d5_kills,d5_deaths,d5_assists,d5_denies,d5_gold,d5_lh,d5_xp,d5_health,d5_max_health,d5_max_mana,d5_level,d5_x,d5_y,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
match_id_hash,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
a400b8f29dece5f4d266f49f1ae2e98a,155,22,7,1,11,11,0,0,0,0,543,7,533,358,600,350.93784,2,116,122,0.000000,0,0,1,0,0.000000,0,0,0,0,78,0,0,0,3,399,4,478,636,720,254.93774,2,124,126,0.000000,0,0,0,0,0.000000,0,0,0,0,14,0,1,0,0,304,0,130,700,700,242.93773,1,70,156,0.000000,0,0,1,0,0.000000,0,0,0,0,59,0,0,0,1,389,4,506,399,700,326.93780,2,170,86,0.000000,0,0,0,0,0.000000,0,0,0,0,77,0,0,0,0,402,10,344,422,800,314.93780,2,120,100,0.000000,0,0,0,0,0.000000,0,0,0,0,12,0,0,1,13,982,12,780,650,720,386.93787,3,82,170,0.000000,0,0,1,0,1.00,0,0,0,0,21,0,0,0,6,788,9,706,640,640,422.93790,3,174,90,0.000000,0,0,2,0,0.00,0,0,0,0,60,0,0,0,1,531,0,307,720,720,242.93773,2,180,84,0.299948,0,0,2,0,0.00,0,0,0,0,84,1,0,0

In [5]:
# --== PREPARE DATA ==--
X_train, y_train, X_test = data_loader.prepare_data(df_train_features.copy(), df_train_targets.copy(), df_test_features.copy())

print(X_train.shape)
X_train.describe()

prepare_data.. Start
prepare_data.. Adding team features
prepare_data.. Replace heroes id
prepare_data.. Drop extra columns: ['game_time', 'game_mode', 'lobby_type', 'objectives_len', 'chat_len']


Prepared data frame: 
       std_kills_ratio  r_mean_kills  d_mean_kills  mean_kills_ratio  \
count     39675.000000  39675.000000  39675.000000      39675.000000   
mean          0.061604      0.202468      0.191507          0.033854   
std           0.055005      0.170373      0.164446          0.033886   
min           0.000000      0.000000      0.000000          0.000000   
25%           0.028271      0.051948      0.048780          0.016317   
50%           0.051111      0.168831      0.158537          0.028189   
75%           0.081119      0.311688      0.292683          0.042161   
max           1.000000      1.000000      1.000000          1.000000   

       std_deaths_ratio  r_mean_deaths  d_mean_deaths  mean_deaths_ratio  \
count      39675.000000   39675.000000   39675.000000  

,std_kills_ratio,r_mean_kills,d_mean_kills,mean_kills_ratio,std_deaths_ratio,r_mean_deaths,d_mean_deaths,mean_deaths_ratio,std_assists_ratio,r_mean_assists,d_mean_assists,mean_assists_ratio,std_denies_ratio,r_mean_denies,d_mean_denies,mean_denies_ratio,std_gold_ratio,r_mean_gold,d_mean_gold,mean_gold_ratio,std_lh_ratio,r_mean_lh,d_mean_lh,mean_lh_ratio,std_xp_ratio,r_mean_xp,d_mean_xp,mean_xp_ratio,std_health_ratio,r_mean_health,d_mean_health,mean_health_ratio,std_max_health_ratio,r_mean_max_health,d_mean_max_health,mean_max_health_ratio,std_max_mana_ratio,r_mean_max_mana,d_mean_max_mana,mean_max_mana_ratio,std_level_ratio,r_mean_level,d_mean_level,mean_level_ratio,std_stuns_ratio,r_mean_stuns,d_mean_stuns,mean_stuns_ratio,std_creeps_stacked_ratio,r_mean_creeps_stacked,d_mean_creeps_stacked,mean_creeps_stacked_ratio,std_camps_stacked_ratio,r_mean_camps_stacked,d_mean_camps_stacked,mean_camps_stacked_ratio,std_rune_pickups_ratio,r_mean_rune_pickups,d_mean_rune_pickups,mean_rune_pickups_ratio,std_firstblood_claimed_ratio,r_mean_firstblood_claimed,d_mean_firstblood_claimed,mean_firstblood_claimed_ratio,std_teamfight_participation_ratio,r_mean_teamfight_participation,d_mean_teamfight_participation,mean_teamfight_participation_ratio,std_towers_killed_ratio,r_mean_towers_killed,d_mean_towers_killed,mean_towers_killed_ratio,std_roshans_killed_ratio,r_mean_roshans_killed,d_mean_roshans_killed,mean_roshans_killed_ratio,std_obs_placed_ratio,r_mean_obs_placed,d_mean_obs_placed,mean_obs_placed_ratio,std_sen_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio,std_distance_ratio,r_mean_distance,d_mean_distance,mean_distance_ratio,r_hero_1,r_hero_10,r_hero_100,r_hero_101,r_hero_102,r_hero_103,r_hero_104,r_hero_105,r_hero_106,r_hero_107,r_hero_108,r_hero_109,r_hero_11,r_hero_110,r_hero_111,r_hero_112,r_hero_113,r_hero_114,r_hero_119,r_hero_12,r_hero_120,r_hero_13,r_hero_14,r_hero_15,r_hero_16,r_hero_17,r_hero_18,r_hero_19,r_hero_2,r_hero_20,r_hero_21,r_hero_22,r_hero_23,r_hero_25,r_hero_26,r_hero_27,r_hero_28,r_hero_29,r_hero_3,r_hero_30,r_hero_31,r_hero_32,r_hero_33,r_hero_34,r_hero_35,r_hero_36,r_hero_37,r_hero_38,r_hero_39,r_hero_4,r_hero_40,r_hero_41,r_hero_42,r_hero_43,r_hero_44,r_hero_45,r_hero_46,r_hero_47,r_hero_48,r_hero_49,r_hero_5,r_hero_50,r_hero_51,r_hero_52,r_hero_53,r_hero_54,r_hero_55,r_hero_56,r_hero_57,r_hero_58,r_hero_59,r_hero_6,r_hero_60,r_hero_61,r_hero_62,r_hero_63,r_hero_64,r_hero_65,r_hero_66,r_hero_67,r_hero_68,r_hero_69,r_hero_7,r_hero_70,r_hero_71,r_hero_72,r_hero_73,r_hero_74,r_hero_75,r_hero_76,r_hero_77,r_hero_78,r_hero_79,r_hero_8,r_hero_80,r_hero_81,r_hero_82,r_hero_83,r_hero_84,r_hero_85,r_hero_86,r_hero_87,r_hero_88,r_hero_89,r_hero_9,r_hero_90,r_hero_91,r_hero_92,r_hero_93,r_hero_94,r_hero_95,r_hero_96,r_hero_97,r_hero_98,r_hero_99,d_hero_1,d_hero_10,d_hero_100,d_hero_101,d_hero_102,d_hero_103,d_hero_104,d_hero_105,d_hero_106,d_hero_107,d_hero_108,d_hero_109,d_hero_11,d_hero_110,d_hero_111,d_hero_112,d_hero_113,d_hero_114,d_hero_119,d_hero_12,d_hero_120,d_hero_13,d_hero_14,d_hero_15,d_hero_16,d_hero_17,d_hero_18,d_hero_19,d_hero_2,d_hero_20,d_hero_21,d_hero_22,d_hero_23,d_hero_25,d_hero_26,d_hero_27,d_hero_28,d_hero_29,d_hero_3,d_hero_30,d_hero_31,d_hero_32,d_hero_33,d_hero_34,d_hero_35,d_hero_36,d_hero_37,d_hero_38,d_hero_39,d_hero_4,d_hero_40,d_hero_41,d_hero_42,d_hero_43,d_hero_44,d_hero_45,d_hero_46,d_hero_47,d_hero_48,d_hero_49,d_hero_5,d_hero_50,d_hero_51,d_hero_52,d_hero_53,d_hero_54,d_hero_55,d_hero_56,d_hero_57,d_hero_58,d_hero_59,d_hero_6,d_hero_60,d_hero_61,d_hero_62,d_hero_63,d_hero_64,d_hero_65,d_hero_66,d_hero_67,d_hero_68,d_hero_69,d_hero_7,d_hero_70,d_hero_71,d_hero_72,d_hero_73,d_hero_74,d_hero_75,d_hero_76,d_hero_77,d_hero_78,d_hero_79,d_hero_8,d_hero_80,d_hero_81,d_hero_82,d_hero_83,d_hero_84,d_hero_85,d_hero_86,d_hero_87,d_hero_88,d_hero_89,d_hero_9,d_hero_90,d_hero_91,d_hero_92,d_hero_93,d_hero_94,d_hero_95,d_hero_96,d_hero_97,d_hero_98,d_hero_99
count,39675.000000,39

In [6]:
X_train.describe()
# X_train = X_train.drop(['r1_kills'], axis=1)
# X_train.shape

,std_kills_ratio,r_mean_kills,d_mean_kills,mean_kills_ratio,std_deaths_ratio,r_mean_deaths,d_mean_deaths,mean_deaths_ratio,std_assists_ratio,r_mean_assists,d_mean_assists,mean_assists_ratio,std_denies_ratio,r_mean_denies,d_mean_denies,mean_denies_ratio,std_gold_ratio,r_mean_gold,d_mean_gold,mean_gold_ratio,std_lh_ratio,r_mean_lh,d_mean_lh,mean_lh_ratio,std_xp_ratio,r_mean_xp,d_mean_xp,mean_xp_ratio,std_health_ratio,r_mean_health,d_mean_health,mean_health_ratio,std_max_health_ratio,r_mean_max_health,d_mean_max_health,mean_max_health_ratio,std_max_mana_ratio,r_mean_max_mana,d_mean_max_mana,mean_max_mana_ratio,std_level_ratio,r_mean_level,d_mean_level,mean_level_ratio,std_stuns_ratio,r_mean_stuns,d_mean_stuns,mean_stuns_ratio,std_creeps_stacked_ratio,r_mean_creeps_stacked,d_mean_creeps_stacked,mean_creeps_stacked_ratio,std_camps_stacked_ratio,r_mean_camps_stacked,d_mean_camps_stacked,mean_camps_stacked_ratio,std_rune_pickups_ratio,r_mean_rune_pickups,d_mean_rune_pickups,mean_rune_pickups_ratio,std_firstblood_claimed_ratio,r_mean_firstblood_claimed,d_mean_firstblood_claimed,mean_firstblood_claimed_ratio,std_teamfight_participation_ratio,r_mean_teamfight_participation,d_mean_teamfight_participation,mean_teamfight_participation_ratio,std_towers_killed_ratio,r_mean_towers_killed,d_mean_towers_killed,mean_towers_killed_ratio,std_roshans_killed_ratio,r_mean_roshans_killed,d_mean_roshans_killed,mean_roshans_killed_ratio,std_obs_placed_ratio,r_mean_obs_placed,d_mean_obs_placed,mean_obs_placed_ratio,std_sen_placed_ratio,r_mean_sen_placed,d_mean_sen_placed,mean_sen_placed_ratio,std_distance_ratio,r_mean_distance,d_mean_distance,mean_distance_ratio,r_hero_1,r_hero_10,r_hero_100,r_hero_101,r_hero_102,r_hero_103,r_hero_104,r_hero_105,r_hero_106,r_hero_107,r_hero_108,r_hero_109,r_hero_11,r_hero_110,r_hero_111,r_hero_112,r_hero_113,r_hero_114,r_hero_119,r_hero_12,r_hero_120,r_hero_13,r_hero_14,r_hero_15,r_hero_16,r_hero_17,r_hero_18,r_hero_19,r_hero_2,r_hero_20,r_hero_21,r_hero_22,r_hero_23,r_hero_25,r_hero_26,r_hero_27,r_hero_28,r_hero_29,r_hero_3,r_hero_30,r_hero_31,r_hero_32,r_hero_33,r_hero_34,r_hero_35,r_hero_36,r_hero_37,r_hero_38,r_hero_39,r_hero_4,r_hero_40,r_hero_41,r_hero_42,r_hero_43,r_hero_44,r_hero_45,r_hero_46,r_hero_47,r_hero_48,r_hero_49,r_hero_5,r_hero_50,r_hero_51,r_hero_52,r_hero_53,r_hero_54,r_hero_55,r_hero_56,r_hero_57,r_hero_58,r_hero_59,r_hero_6,r_hero_60,r_hero_61,r_hero_62,r_hero_63,r_hero_64,r_hero_65,r_hero_66,r_hero_67,r_hero_68,r_hero_69,r_hero_7,r_hero_70,r_hero_71,r_hero_72,r_hero_73,r_hero_74,r_hero_75,r_hero_76,r_hero_77,r_hero_78,r_hero_79,r_hero_8,r_hero_80,r_hero_81,r_hero_82,r_hero_83,r_hero_84,r_hero_85,r_hero_86,r_hero_87,r_hero_88,r_hero_89,r_hero_9,r_hero_90,r_hero_91,r_hero_92,r_hero_93,r_hero_94,r_hero_95,r_hero_96,r_hero_97,r_hero_98,r_hero_99,d_hero_1,d_hero_10,d_hero_100,d_hero_101,d_hero_102,d_hero_103,d_hero_104,d_hero_105,d_hero_106,d_hero_107,d_hero_108,d_hero_109,d_hero_11,d_hero_110,d_hero_111,d_hero_112,d_hero_113,d_hero_114,d_hero_119,d_hero_12,d_hero_120,d_hero_13,d_hero_14,d_hero_15,d_hero_16,d_hero_17,d_hero_18,d_hero_19,d_hero_2,d_hero_20,d_hero_21,d_hero_22,d_hero_23,d_hero_25,d_hero_26,d_hero_27,d_hero_28,d_hero_29,d_hero_3,d_hero_30,d_hero_31,d_hero_32,d_hero_33,d_hero_34,d_hero_35,d_hero_36,d_hero_37,d_hero_38,d_hero_39,d_hero_4,d_hero_40,d_hero_41,d_hero_42,d_hero_43,d_hero_44,d_hero_45,d_hero_46,d_hero_47,d_hero_48,d_hero_49,d_hero_5,d_hero_50,d_hero_51,d_hero_52,d_hero_53,d_hero_54,d_hero_55,d_hero_56,d_hero_57,d_hero_58,d_hero_59,d_hero_6,d_hero_60,d_hero_61,d_hero_62,d_hero_63,d_hero_64,d_hero_65,d_hero_66,d_hero_67,d_hero_68,d_hero_69,d_hero_7,d_hero_70,d_hero_71,d_hero_72,d_hero_73,d_hero_74,d_hero_75,d_hero_76,d_hero_77,d_hero_78,d_hero_79,d_hero_8,d_hero_80,d_hero_81,d_hero_82,d_hero_83,d_hero_84,d_hero_85,d_hero_86,d_hero_87,d_hero_88,d_hero_89,d_hero_9,d_hero_90,d_hero_91,d_hero_92,d_hero_93,d_hero_94,d_hero_95,d_hero_96,d_hero_97,d_hero_98,d_hero_99
count,39675.000000,39

### First model

Training function

In [7]:
def train_model(X, X_test, y, params, folds, model_type='lgb', plot_feature_importance=False, averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.loc[train_index], X.loc[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        
        if model_type == 'lgb':
            train_data = lgb.Dataset(X_train, label=y_train)
            valid_data = lgb.Dataset(X_valid, label=y_valid)
            
            model = lgb.train(params,
                    train_data,
                    num_boost_round=20000,
                    valid_sets = [train_data, valid_data],
                    verbose_eval=1000,
                    early_stopping_rounds = 200)
            
            y_pred_valid = model.predict(X_valid)
            y_pred = model.predict(X_test, num_iteration=model.best_iteration)
            
        if model_type == 'xgb':
            train_data = xgb.DMatrix(data=X_train, label=y_train, feature_names=X_train.columns)
            valid_data = xgb.DMatrix(data=X_valid, label=y_valid, feature_names=X_train.columns)

            watchlist = [(train_data, 'train'), (valid_data, 'valid_data')]
            model = xgb.train(dtrain=train_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200, verbose_eval=500, params=params)
            y_pred_valid = model.predict(xgb.DMatrix(X_valid, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
            y_pred = model.predict(xgb.DMatrix(X_test, feature_names=X_train.columns), ntree_limit=model.best_ntree_limit)
        
        if model_type == 'sklearn':
            model = model
            model.fit(X_train, y_train)
            y_pred_valid = model.predict_proba(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            # print(f'Fold {fold_n}. AUC: {score:.4f}.')
            # print('')
            
            y_pred = model.predict_proba(X_test)[:, 1]
            
        if model_type == 'glm':
            model = sm.GLM(y_train, X_train, family=sm.families.Binomial())
            model_results = model.fit()
            model_results.predict(X_test)
            y_pred_valid = model_results.predict(X_valid).reshape(-1,)
            score = roc_auc_score(y_valid, y_pred_valid)
            
            y_pred = model_results.predict(X_test)
            
        if model_type == 'cat':
            model = CatBoostClassifier(iterations=20000, learning_rate=0.05, loss_function='Logloss',  eval_metric='AUC', **params)
            model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=[], use_best_model=True, verbose=False)

            y_pred_valid = model.predict_proba(X_valid)[:, 1]
            y_pred = model.predict_proba(X_test)[:, 1]
            
        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  
        
        if model_type == 'lgb':
            # feature importance
            fold_importance = pd.DataFrame()
            fold_importance["feature"] = X.columns
            fold_importance["importance"] = model.feature_importance()
            fold_importance["fold"] = fold_n + 1
            feature_importance = pd.concat([feature_importance, fold_importance], axis=0)

    prediction /= n_fold
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    if model_type == 'lgb':
        feature_importance["importance"] /= n_fold
        if plot_feature_importance:
            cols = feature_importance[["feature", "importance"]].groupby("feature").mean().sort_values(
                by="importance", ascending=False)[:50].index

            best_features = feature_importance.loc[feature_importance.feature.isin(cols)]

            plt.figure(figsize=(16, 12));
            sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False));
            plt.title('LGB Features (avg over folds)');
        
            return oof, prediction, feature_importance
        return oof, prediction, scores
    
    else:
        return oof, prediction, scores
    
    
def lgb_model_tunning(X, y, params):
    print('lgb_model_tunning... Start')

    # Create parameters to search
    gridParams = {
        'learning_rate': [0.005, 0.01],
        'n_estimators': [40],
        'num_leaves': [12, 16, 32, 64],
        'boosting_type': ['gbdt'],
        'objective': ['binary'],
        'random_state': [SEED],  # Updated from 'seed'
        'colsample_bytree': [0.65, 0.66],
        'subsample': [0.7, 0.75],
        'reg_alpha': [1, 1.2],
        'reg_lambda': [1, 1.2, 1.4],
    }

    # Create classifier to use. Note that parameters have to be input manually
    # not as a dict!
    mdl = lgb.LGBMClassifier(boosting_type='gbdt',
                             objective='binary',
                             n_jobs=4,  # Updated from 'nthread'
                             silent=True,
                             max_depth=params['max_depth'],
                             # max_bin=params['max_bin'],
                             # subsample_for_bin=params['subsample_for_bin'],
                             # subsample=params['subsample'],
                             # subsample_freq=params['subsample_freq'],
                             # min_split_gain=params['min_split_gain'],
                             # min_child_weight=params['min_child_weight'],
                             # min_child_samples=params['min_child_samples'],
                             # scale_pos_weight=params['scale_pos_weight']
                             )

    # To view the default model params:
    print(mdl.get_params().keys())

    # Create the grid
    grid = GridSearchCV(mdl, gridParams,
                        verbose=0,
                        cv=4,
                        n_jobs=4)
    # Run the grid
    grid.fit(X, y)

    # Print the best parameters found
    print('===== Best params =====')
    print(grid.best_params_)
    print(grid.best_score_)

    # Using parameters already set above, replace in the best from the grid search
    params['colsample_bytree'] = grid.best_params_['colsample_bytree']
    params['learning_rate'] = grid.best_params_['learning_rate']
    # params['max_bin'] = grid.best_params_['max_bin']
    params['num_leaves'] = grid.best_params_['num_leaves']
    params['reg_alpha'] = grid.best_params_['reg_alpha']
    params['reg_lambda'] = grid.best_params_['reg_lambda']
    params['subsample'] = grid.best_params_['subsample']
    # params['subsample_for_bin'] = grid.best_params_['subsample_for_bin']

    print('Fitting with params: ')
    print(params)
    return params

n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

print('added')

added


In [11]:
params = {'boost': 'gbdt',
          'feature_fraction': 0.05,
          'learning_rate': 0.01,
          'max_depth': -1,  
          'metric':'auc',
          'min_data_in_leaf': 50,
          'num_leaves': 32,
          'num_threads': -1,
          'verbosity': 1,
          'objective': 'binary'
         }

tunning = True
if tunning:
    params = lgb_model_tunning(X_train, y_train, params)

oof_lgb, prediction_lgb, scores = train_model(X_train, X_test, y_train, params=params, folds=folds, model_type='lgb', plot_feature_importance=True)

lgb_model_tunning... Start
dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample', 'subsample_for_bin', 'subsample_freq'])


KeyboardInterrupt: 

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['radiant_win_prob'] = prediction_lgb
sub.to_csv('submission.csv', index=False)
sub.head()